# การทดลองเสริม: การไล่ระดับ (Gradient Descent) สำหรับการถดถอยโลจิสติก (Logistic Regression)


## เป้าหมายของการทดลองนี้
ในห้องปฏิบัติการนี้ คุณจะ:
- อัปเดตอัลกอริทึมการไล่ระดับ (gradient descent) สำหรับการถดถอยโลจิสติก (logistic regression)
- สำรวจการทำงานของอัลกอริทึมการไล่ระดับบนชุดข้อมูลที่คุ้นเคย

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
try:
  %matplotlib widget
  print("widget is already installed")
except:
  print("widget is not been installed, install now..")
  !pip install ipympl

In [ ]:
# prompt: import requsts and download from this github link : https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/deeplearning.mplstyle

import requests
from pathlib import Path

url = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/deeplearning.mplstyle'
url2 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/plt_quad_logistic.py'
url3 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/lab_utils_common_c3.py'

response = requests.get(url)
with open('deeplearning.mplstyle', 'wb') as f:
  f.write(response.content)

response = requests.get(url2)
with open('plt_quad_logistic.py', 'wb') as f:
  f.write(response.content)

response = requests.get(url3)
with open('lab_utils_common.py', 'wb') as f:
  f.write(response.content)

In [ ]:
import copy, math
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from lab_utils_common import  dlc, plot_data, plt_tumor_data, sigmoid, compute_cost_logistic
from plt_quad_logistic import plt_quad_logistic, plt_prob
plt.style.use('./deeplearning.mplstyle')

## ชุดข้อมูล (Data set)
เริ่มต้นด้วยชุดข้อมูลสองฟีเจอร์ (features) เดียวกันกับที่ใช้ในห้องปฏิบัติการการตัดสินใจ (decision boundary lab)

In [ ]:
X_train = np.array([[0.5, 1.5], [1,1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])
y_train = np.array([0, 0, 0, 1, 1, 1])

เราจะใช้ฟังก์ชันช่วยในการพล็อตข้อมูลนี้ จุดข้อมูลที่มีป้ายกำกับ $y=1$ จะแสดงเป็นกากบาทสีแดง ขณะที่จุดข้อมูลที่มีป้ายกำกับ $y=0$ จะแสดงเป็นวงกลมสีน้ำเงิน

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,4))
plot_data(X_train, y_train, ax)

ax.axis([0, 4, 0, 3.5])
ax.set_ylabel('$x_1$', fontsize=12)
ax.set_xlabel('$x_0$', fontsize=12)
plt.show()

## Logistic Gradient Descent
<img align="right" src="./images/C1_W3_Logistic_gradient_descent_v1.png"     style=" width:400px; padding: 10px; " >

อัลกอริทึมการไล่ระดับ (gradient descent algorithm) ใช้การคำนวณการไล่ระดับ (gradient calculation)

$$\begin{align*}
&\text{repeat until convergence:} \; \lbrace \\
&  \; \; \;w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1}  \; & \text{for j := 0..n-1} \\ 
&  \; \; \;  \; \;b = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b} \\
&\rbrace
\end{align*}$$

ในแต่ละการวนซ้ำ (iteration) จะทำการอัปเดตพารามิเตอร์ $w_j$ พร้อมกันสำหรับทุกค่า $j$

$$\begin{align*}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{2} \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{3} 
\end{align*}$$

* m คือจำนวนตัวอย่างในการฝึกทั้งหมดในชุดข้อมูล
* $f_{\mathbf{w},b}(x^{(i)})$ คือการทำนายของโมเดล ขณะที่ $y^{(i)}$ คือเป้าหมาย
* ในกรณีของโมเดลการถดถอยโลจิสติก (logistic regression):
    $z = \mathbf{w} \cdot \mathbf{x} + b$  
    $f_{\mathbf{w},b}(x) = g(z)$  
    โดยที่ $g(z)$ คือ sigmoid function:  
    $g(z) = \frac{1}{1+e^{-z}}$   
    


### การใช้งานอัลกอริทึมการไล่ระดับ (Gradient Descent Implementation)
อัลกอริทึมการไล่ระดับ (gradient descent) ประกอบด้วยสองส่วนหลัก:
- ลูปการคำนวณสมการ (1): ส่วนนี้มักจะถูกกำหนดให้คุณในแบบฝึกหัดหรือตัวอย่าง
- การคำนวณ gradient ณ จุดปัจจุบัน (current gradient): ส่วนนี้คุณจะต้องเขียนโค้ดเองในแบบฝึกหัดสัปดาห์นี้

#### การคำนวณ Gradient: คำอธิบายโค้ด

โค้ดนี้จะทำการคำนวณสมการ (2) และ (3) สำหรับทุกค่า $w_j$ และ $b$.
มีหลายวิธีในการเขียนโค้ดนี้ นี่คือหนึ่งในวิธี:
- เริ่มต้นตัวแปร: สร้างตัวแปร `dj_dw` และ `dj_db`  เพื่อเก็บผลรวม
- วนลูปสำหรับแต่ละตัวอย่าง
    - คำนวณค่าผิดพลาด (error) สำหรับตัวอย่างที่  $g(\mathbf{w} \cdot \mathbf{x}^{(i)} + b) - \mathbf{y}^{(i)}$
    - สำหรับแต่ละค่า input $x_{j}^{(i)}$ ในตัวอย่างนี้:
        - คูณค่าผิดพลาดด้วย   $x_{j}^{(i)}$, แล้วบวกเข้ากับค่าที่สอดคล้องกันใน `dj_dw`. (สมการ 2)
    - บวกค่าผิดพลาดเข้ากับ `dj_db` (สมการ 3)

- หาร  `dj_db` และ `dj_dw` ด้วยจำนวนตัวอย่างทั้งหมด (m)
- หมายเหตุ $\mathbf{x}^{(i)}$ ใน numpy คือ  `X[i,:]` หรือ  `X[i]`  และ $x_{j}^{(i)}$ คือ `X[i,j]`

In [ ]:
def compute_gradient_logistic(X, y, w, b): 
    """
    Computes the gradient for logistic regression 
 
    Args:
      X (ndarray (m,n): Data, m examples with n features
      y (ndarray (m,)): target values
      w (ndarray (n,)): model parameters  
      b (scalar)      : model parameter
    Returns
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w. 
      dj_db (scalar)      : The gradient of the cost w.r.t. the parameter b. 
    """
    m,n = X.shape
    dj_dw = np.zeros((n,))                           #(n,)
    dj_db = 0.

    for i in range(m):
        f_wb_i = sigmoid(np.dot(X[i],w) + b)          #(n,)(n,)=scalar
        err_i  = f_wb_i  - y[i]                       #scalar
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err_i * X[i,j]      #scalar
        dj_db = dj_db + err_i
    dj_dw = dj_dw/m                                   #(n,)
    dj_db = dj_db/m                                   #scalar
        
    return dj_db, dj_dw  

ตรวจสอบการใช้งานฟังก์ชันการไล่ระดับ (gradient function) โดยใช้เซลล์ด้านล่าง

In [ ]:
X_tmp = np.array([[0.5, 1.5], [1,1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])
y_tmp = np.array([0, 0, 0, 1, 1, 1])
w_tmp = np.array([2.,3.])
b_tmp = 1.
dj_db_tmp, dj_dw_tmp = compute_gradient_logistic(X_tmp, y_tmp, w_tmp, b_tmp)
print(f"dj_db: {dj_db_tmp}" )
print(f"dj_dw: {dj_dw_tmp.tolist()}" )

**ผลลัพธ์ที่คาดหวัง**
``` 
dj_db: 0.49861806546328574
dj_dw: [0.498333393278696, 0.49883942983996693]
```

#### โค้ดการไล่ระดับ (Gradient Descent Code)
โค้ดที่นำมาใช้ในการคำนวณสมการ (1) ด้านบน มีการนำมาใช้งานดังนี้

โปรดสักครู่เพื่อค้นหาและเปรียบเทียบฟังก์ชันต่างๆ ภายในรูทีนกับสมการด้านบน

In [ ]:
def gradient_descent(X, y, w_in, b_in, alpha, num_iters): 
    """
    Performs batch gradient descent
    
    Args:
      X (ndarray (m,n)   : Data, m examples with n features
      y (ndarray (m,))   : target values
      w_in (ndarray (n,)): Initial values of model parameters  
      b_in (scalar)      : Initial values of model parameter
      alpha (float)      : Learning rate
      num_iters (scalar) : number of iterations to run gradient descent
      
    Returns:
      w (ndarray (n,))   : Updated values of parameters
      b (scalar)         : Updated value of parameter 
    """
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in
    
    for i in range(num_iters):
        # Calculate the gradient and update the parameters
        dj_db, dj_dw = compute_gradient_logistic(X, y, w, b)   

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw               
        b = b - alpha * dj_db               
      
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append( compute_cost_logistic(X, y, w, b) )

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]}   ")
        
    return w, b, J_history         #return final w,b and J history for graphing


มาลองรันอัลกอริทึมการไล่ระดับ (gradient descent) กับชุดข้อมูลของเรากัน

In [ ]:
w_tmp  = np.zeros_like(X_train[0])
b_tmp  = 0.
alph = 0.1
iters = 10000

w_out, b_out, _ = gradient_descent(X_train, y_train, w_tmp, b_tmp, alph, iters) 
print(f"\nupdated parameters: w:{w_out}, b:{b_out}")

#### มาพล็อตผลลัพธ์ของการไล่ระดับกันเถอะ


In [ ]:
fig,ax = plt.subplots(1,1,figsize=(5,4))
# plot the probability 
plt_prob(ax, w_out, b_out)

# Plot the original data
ax.set_ylabel(r'$x_1$')
ax.set_xlabel(r'$x_0$')   
ax.axis([0, 4, 0, 3.5])
plot_data(X_train,y_train,ax)

# Plot the decision boundary
x0 = -b_out/w_out[0]
x1 = -b_out/w_out[1]
ax.plot([0,x0],[x1,0], c=dlc["dlblue"], lw=1)
plt.show()

ในพล็อตข้างต้น:
- การแรเงา (shading) แสดงถึงความน่าจะเป็นของ y=1 (ผลลัพธ์ก่อนการตัดสินใจ)
- เส้นตัดสินใจ (decision boundary) คือเส้นที่ความน่าจะเป็น = 0.5

## อีกหนึ่งชุดข้อมูล: การแสดงภาพค่าใช้จ่าย (Cost Function)
กลับมาที่ชุดข้อมูลตัวแปรเดียวกัน
- ด้วยเพียงสองพารามิเตอร์ $w$ และ $b$ เราสามารถพล็อตฟังก์ชันค่าใช้จ่าย (Cost Function) เป็นแผนที่ Contour เพื่อให้เห็นภาพที่ดีขึ้นเกี่ยวกับการทำงานของอัลกอริทึมการไล่ระดับ (gradient descent)





In [ ]:
x_train = np.array([0., 1, 2, 3, 4, 5])
y_train = np.array([0,  0, 0, 1, 1, 1])

แสดงข้อมูลด้วยฟังก์ชันช่วย
เราจะใช้ฟังก์ชันช่วยในการพล็อตข้อมูลนี้

- ข้อมูลที่มีค่า y=1 จะแสดงเป็นเครื่องหมายกากบาทสีแดง
- ข้อมูลที่มีค่า y=0 จะแสดงเป็นวงกลมสีน้ำเงิน







In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,3))
plt_tumor_data(x_train, y_train, ax)
plt.show()

การทดลองกับกราฟ: เปลี่ยนค่า w และ b
คำอธิบาย:

- ในกราฟด้านล่าง คุณสามารถลองเปลี่ยนค่า w และ b โดยคลิกภายในแผนภูมิ Contour (contour plot) ที่ด้านบนขวา
    - การเปลี่ยนแปลงอาจใช้เวลาหนึ่งหรือสองวินาที
    - สังเกตการเปลี่ยนแปลงของค่าใช้จ่าย (cost) ที่พล็อตด้านบนซ้าย
    - ค่าใช้จ่ายจะสะสมโดยการสูญเสีย (loss) ในแต่ละตัวอย่าง (เส้นประตั้ง)
- คุณสามารถเรียกใช้อัลกอริทึมการไล่ระดับ (gradient descent) โดยคลิกปุ่มสีส้ม
    - สังเกตการลดลงของค่าใช้จ่ายอย่างสม่ำเสมอ (แผนภูมิ Contour และแผนภูมิค่าใช้จ่ายอยู่ในสเกล log(cost))
- หากต้องการรีเซ็ตกราฟ ให้เรียกเซลล์ใหม่ (rerun the cell)


In [ ]:
w_range = np.array([-1, 7])
b_range = np.array([1, -14])
quad = plt_quad_logistic( x_train, y_train, w_range, b_range )

## ขอแสดงความยินดี!
คุณได้:
- ศึกษาสูตรและการใช้งานในการคำนวณการไล่ระดับ (gradient) สำหรับการถดถอยโลจิสติก (logistic regression)
- นำรูทีนเหล่านั้นไปใช้งานใน:
    - การสำรวจชุดข้อมูลตัวแปรเดียว
    - การสำรวจชุดข้อมูลตัวแปรสองตัว






